In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Activation, Conv2DTranspose
from keras.models import Model, Sequential, load_model
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

% matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

Using TensorFlow backend.


In [2]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))

In [3]:
adam = Adam(lr=0.0002, beta_1=0.5)
def keras_generator():
  # use tutorial from https://medium.com/towards-data-science/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0
  g = Sequential()
  dropout = 0.4
  depth = 256
  dim = 7
  # In: 100
  # Out: dim x dim x depth
  g.add(Dense(dim * dim * depth, input_dim=100))
  g.add(BatchNormalization(momentum=0.9))
  g.add(Activation('relu'))
  g.add(Reshape((dim, dim, depth)))
  g.add(Dropout(dropout))
  # In: dim x dim x depth
  # Out: 2*dim x 2*dim x depth/2
  g.add(UpSampling2D())
  g.add(Conv2DTranspose(int(depth / 2), 5, padding='same'))
  g.add(BatchNormalization(momentum=0.9))
  g.add(Activation('relu'))
  g.add(UpSampling2D())
  g.add(Conv2DTranspose(int(depth / 4), 5, padding='same'))
  g.add(BatchNormalization(momentum=0.9))
  g.add(Activation('relu'))
  # Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix
  g.add(Conv2DTranspose(1, 5, padding='same'))
  g.add(Activation('sigmoid'))
  g.compile(loss='binary_crossentropy', optimizer=adam)
  return g

In [4]:
g = keras_generator()
g.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_1 (Batch (None, 12544)             50176     
_________________________________________________________________
activation_1 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 128)       819328    
__________

In [5]:
d = load_model(os.path.join('./discriminator-learn/learned_model/discriminator.h5'))
d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
__________

In [6]:
randomDim = 100
ganInput = Input(shape=(randomDim,))

x = g(ganInput)
y = d(x)

gan = Model(inputs=ganInput, outputs=y)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [7]:
dLosses = []
gLosses = []

In [8]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/gan_loss_epoch_%d.png' % epoch)

In [9]:
# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/gan_generated_image_epoch_%d.png' % epoch)

In [10]:
# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    g.save('./gan_generator_epoch_%d.h5' % epoch)
    d.save('./gan_discriminator_epoch_%d.h5' % epoch)

In [11]:
def train(epochs=1, batchSize=128):
    batchCount = int(X_train.shape[0] / batchSize)

    for e in range(1, epochs+1):
        for _ in range(batchCount):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            # Generate fake MNIST images
            generatedImages = g.predict(noise)
            # imageBatch = imageBatch.reshape((100352, 28, 28, 1))
            print(np.shape(imageBatch), np.shape(generatedImages))
            X = np.concatenate([imageBatch, generatedImages])

            # Labels for generated and real data
            yDis = np.zeros(2*batchSize)
            # One-sided label smoothing
            yDis[:batchSize] = 0.9

            # Train discriminator
            d.trainable = True
            dloss = d.train_on_batch(X, yDis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            d.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        # Store loss of most recent batch from this epoch
        dLosses.append(dloss)
        gLosses.append(gloss)

        if e == 1 or e % 20 == 0:
            plotGeneratedImages(e)
            saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)


In [12]:
train(200, 128)

(128, 28, 28, 1) (128, 28, 28, 1)


ValueError: Error when checking target: expected dense_2 to have shape (None, 10) but got array with shape (256, 1)